<a href="https://colab.research.google.com/github/MAN1108/github-test/blob/main/GPT-3.5-turbo-RAG-wwith_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain openai weaviate-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [3]:
import openai
import os # Or the relevant LangChain modules
os.environ["OPENAI_API_KEY"] = "sk-proj-idvBS8JFLJQI1Hal5pM3T3BlbkFJzuJU3E7804sPS1HNhMeP"

In [4]:
!pip install python-dotenv
import dotenv
dotenv.load_dotenv()

False

In [5]:
!pip install pypdf
from langchain.document_loaders import PyPDFLoader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 7.5 MB/s eta 0:00:00


In [6]:
def load_pdf_files(directory_path):
    """Loads text from multiple PDFs in a directory."""
    documents = []
    for root, _, files in os.walk(directory_path):
        for file in files:
            if file.endswith('.pdf'):
                file_path = os.path.join(root, file)
                loader = PyPDFLoader(file_path)
                documents.extend(loader.load())
    return documents

directory_path = "/content/sample_data/harry"  # Replace with your directory
documents = load_pdf_files(directory_path)
print(len(documents))


3917


In [7]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

In [8]:
!pip install tiktoken
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions

client = weaviate.Client(
  embedded_options = EmbeddedOptions()
)

vectorstore = Weaviate.from_documents(
    client = client,
    documents = chunks,
    embedding = OpenAIEmbeddings(api_key="sk-proj-idvBS8JFLJQI1Hal5pM3T3BlbkFJzuJU3E7804sPS1HNhMeP"),
    by_text = False
)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.1 MB/s eta 0:00:00
Binary /root/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.23.7/weaviate-v1.23.7-Linux-amd64.tar.gz
Started /root/.cache/weaviate-embedded: process ID 1564


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [9]:
retriever = vectorstore.as_retriever()

In [10]:
from langchain.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
just give one-shot answer.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question. \nIf you don't know the answer, just say that you don't know. \njust give one-shot answer.\nQuestion: {question} \nContext: {context} \nAnswer:\n"))]


In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

query = "What is the name of the shop where students buy their school supplies for Hogwarts"
rag_chain.invoke(query)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


'Quality Quidditch Supplies'

In [19]:
!pip install datasets
from datasets import Dataset

questions = ["Which Quidditch players fend off the Bludgers?",
             "What is the Chudley Cannons' team colour?",
             "Where could you buy broomsticks?",
             "In which place is the Hogwarts Sorting Hat ceremony held?",
             "Dittany is used primarily for what purpose?",
             "What is the name of the ghost who haunts the Ravenclaw common room?",
             "What is the name of the spell used to create a protective shield around the caster?",
             "What is the name of the potion that gives good luck to the drinker?",
             "What is the name of Slytherin's ghost?",
             "What is the name of the wand shop in Diagon Alley?",
             "Why does Horace Slughorn collect famous and talented wizards in 'Harry Potter and the Half-Blood Prince'?",
             "When Riding Facing Backwards, how does steering work?",
             "How does Sirius Blacks mentorship shape Harrys understanding of family and belonging?",
             "Why does Harry start hearing voices in 'Harry Potter and the Chamber of Secrets'?",
             "What is the significance of Harrys friendship with Luna Lovegood?",
             "Why does Moaning Myrtle haunt the girls bathroom in 'Harry Potter and the Chamber of Secrets'?",
             "Why does Harry become an Auror after Hogwarts in 'Harry Potter and the Deathly Hallows'?",
             "Why does Fleur Delacour fall in love with Bill Weasley in 'Harry Potter and the Half-Blood Prince'?",
             "How does Dumbledores relationship with Harry influence his perception of power and responsibility?",
             "Why does Harry receive a Firebolt broomstick in 'Harry Potter and the Prisoner of Azkaban'?",

            ]
ground_truths = [["The Beaters"],
                 ["Orange"],
                 ["Quality Quidditch Supplies"],
                 ["The Great Hall at Hogwarts"],
                 ["Healing"],
                 ["The Grey Lady"],
                 ["Protego"],
                 ["Liquid Luck, also known as Felix Felicis"],
                 ["The Bloody Baron."],
                 ["Ollivander's."],
                 ["For his Slug Club, to network and be associated with successful individuals."],
                 ["Turn to the opposite direction"],
                 ["It helps Harry find a sense of belonging and acceptance within the wizarding world, as he sees Sirius as a father figure and role model."],
                 ["Because he is a Parselmouth and can understand snakes."],
                 ["It highlights the importance of accepting others for who they are and embracing diversity."],
                 ["Because she died there after looking into the eyes of the Basilisk."],
                 ["To help maintain peace in the wizarding world."],
                 ["Because of his kindness, bravery, and support during the Triwizard Tournament"],
                 ["It teaches him that true power comes from compassion and empathy, and that with great power comes great responsibility."],
                 ["As a gift from Sirius Black and to replace his destroyed broom."],
                ]
answers = []
contexts = []

# Inference
for query in questions:
  answers.append(rag_chain.invoke(query))
  contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

In [20]:
dataset.to_csv("gpt 3.5 turbo-RAG_dataset.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

147805

In [28]:
import pandas as pd
test_df = pd.read_csv("/content/sample_data/gpt 3.5 turbo-RAG_dataset_without)context.csv")

In [31]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

ground_truth = test_df['ground_truths'].tolist()
predictions = test_df['answer'].tolist()


def calculate_metrics(ground_truth, predictions):
  """
  Calculates and prints various evaluation metrics for question answering.

  Args:
      ground_truth: A list of strings containing the correct answers.
      predictions:  A list of strings containing the model's predicted answers.

  Returns:
      A dictionary containing the calculated metrics (accuracy, precision, recall, F1-score).
  """
  metrics = {}

  # Ensure equal list lengths (handle cases with missing predictions)
  ground_truth = ground_truth[:len(predictions)]
  predictions = predictions[:len(ground_truth)]
  predictions = [pred if pred is not None else "I'm not sure" for pred in predictions]
  # Calculate metrics
  metrics["accuracy"] = accuracy_score(ground_truth, predictions)
  metrics["precision"] = precision_score(ground_truth, predictions, average='weighted')  # Weighted precision for imbalanced datasets
  metrics["recall"] = recall_score(ground_truth, predictions, average='weighted')  # Weighted recall for imbalanced datasets
  metrics["f1"] = f1_score(ground_truth, predictions, average='weighted')  # Weighted F1-score for imbalanced datasets

  # Print Metric Results
  print("Evaluation Metrics:")
  for metric_name, value in metrics.items():
    print(f"{metric_name}: {value:.4f}")

  return metrics

# ... (Your code for generating ground_truth and predictions) ...

metrics = calculate_metrics(ground_truth, predictions)

Evaluation Metrics:
accuracy: 0.0000
precision: 0.0000
recall: 0.0000
f1: 0.0000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [32]:
!pip install ragas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 10.0 MB/s eta 0:00:00


In [43]:
eval_df = pd.read_csv("/content/sample_data/gpt 3.5 turbo-RAG_dataset.csv")
eval_df1=eval_df
display(eval_df.head())

,question,contexts,answer,ground_truth
0,Which Quidditch players fend off the Bludgers?,"['""Three Chasers, one Keeper,"" said Harry, who...",Beaters are the Quidditch players who fend off...,The Beaters
1,What is the Chudley Cannons' team colour?,['Seven green blurs swept onto the field; Harr...,The Chudley Cannons' team colour is green.,Orange
2,Where could you buy broomsticks?,"['""Irish International Side\'s Just put in an ...",You could buy broomsticks at Quality Quidditch...,Quality Quidditch Supplies
3,In which place is the Hogwarts Sorting Hat cer...,['Innumerable candles were hovering in midair ...,The Hogwarts Sorting Hat ceremony is held in t...,The Great Hall at Hogwarts
4,Dittany is used primarily for what purpose?,"['The Thief\nHe met Hermione, also on her hand...",Dittany is primarily used for healing wounds a...,Healing


In [48]:
from datasets import Dataset
def assemble_ragas_dataset(input_df):
    question_list, truth_list, context_list = [], [], []
    question_list = input_df.question.astype(str).to_list()
    truth_list = eval_df.ground_truth.astype(str).to_list()
    context_list = input_df.contexts.astype(str).to_list()
    context_list = [[context] for context in context_list]
    rag_answer_list = input_df.astype(str).answer.to_list()
    ragas_ds = Dataset.from_dict({"question": question_list,
                            "contexts": context_list,
                            "answer": rag_answer_list,
                            "ground_truth": truth_list
                            })
    return ragas_ds
ragas_input_ds = assemble_ragas_dataset(eval_df)
display(ragas_input_ds)

Dataset({
    features: ['question', 'contexts', 'answer', 'ground_truth'],
    num_rows: 20
})

In [46]:
import os, openai, pprint
from openai import OpenAI

# Save the api key in an env variable.
os.environ['OPENAI_API_KEY'] = 'sk-proj-idvBS8JFLJQI1Hal5pM3T3BlbkFJzuJU3E7804sPS1HNhMeP'

# Choose the metrics you want to see.
from ragas.metrics import ( context_recall, context_precision, faithfulness, )
metrics = ['context_recall', 'context_precision', 'faithfulness']

# Change the llm-as-critic.
from ragas.llms import llm_factory
LLM_NAME = "gpt-3.5-turbo"
ragas_llm = llm_factory(model=LLM_NAME)

# Also change the embeddings.
from langchain_openai.embeddings import OpenAIEmbeddings
from ragas.embeddings import LangchainEmbeddingsWrapper
lc_embeddings = OpenAIEmbeddings( model="text-embedding-3-small", dimensions=512 )
ragas_emb = LangchainEmbeddingsWrapper(embeddings=lc_embeddings)

# Change the default models used for each metric.
for metric in metrics:
    globals()[metric].llm = ragas_llm
    globals()[metric].embeddings = ragas_emb

# Evaluate the dataset.
from ragas import evaluate
ragas_result = evaluate( ragas_input_ds,
    metrics=[ context_precision, context_recall, faithfulness, ],
    llm=ragas_llm,
)

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

In [49]:
ragas_output_df = ragas_result.to_pandas()
ragas_output_df

,question,contexts,answer,ground_truth,context_precision,context_recall,faithfulness
0,Which Quidditch players fend off the Bludgers?,"[['""Three Chasers, one Keeper,"" said Harry, wh...",Beaters are the Quidditch players who fend off...,The Beaters,1.0,0.666667,1.0
1,What is the Chudley Cannons' team colour?,[['Seven green blurs swept onto the field; Har...,The Chudley Cannons' team colour is green.,Orange,0.0,0.000000,0.0
2,Where could you buy broomsticks?,"[['""Irish International Side\'s Just put in an...",You could buy broomsticks at Quality Quidditch...,Quality Quidditch Supplies,0.0,1.000000,1.0
3,In which place is the Hogwarts Sorting Hat cer...,[['Innumerable candles were hovering in midair...,The Hogwarts Sorting Hat ceremony is held in t...,The Great Hall at Hogwarts,1.0,1.000000,1.0
4,Dittany is used primarily for what purpose?,"[['The Thief\nHe met Hermione, also on her han...",Dittany is primarily used for healing wounds a...,Healing,1.0,1.000000,1.0
5,What is the name of the ghost who haunts the R...,[['The Battle of Hogwarts\ndiadem that had elu...,The ghost who haunts the Ravenclaw common room...,The Grey Lady,1.0,1.000000,1.0
6,What is the name of the spell used to create a...,[['THE HALF-BLOOD PRINCE \n\x91 179 \x91 of ...,The spell used to create a protective shield a...,Protego,1.0,0.000000,1.0
7,What is the name of the potion that gives good...,[['THE HALF-BLOOD PRINCE \n\x91 187 \x91 col...,The name of the potion that gives good luck to...,"Liquid Luck, also known as Felix Felici",1.0,1.000000,1.0
8,What is the name of Slytherin's ghost?,[['CHAPTER TWENTY-ONE \n\x91 460 \x91 recall ...,Nearly Headless Nick,The Bloody Baron.,0.0,0.600000,0.0
9,What is the name of the wand shop in Diagon Al...,[['The last shop was narrow and shabby. Peelin...,Ollivanders: Makers of Fine Wands since 382 B.C.,Ollivander's.,1.0,0.500000,1.0
